# PageRank: Google's own Prioritising Algorithm

We first define the `DAMPING` and `SAMPLES`, the number of samples we’ll use to estimate PageRank using the sampling method, initially set to 10,000 samples.

And obviously import the required libraries.

Before you get started with the Assignment, You may want to look a bit into the `random` module (as well as the `numpy.random` module, if you plan to use Numpy!).





In [1]:
import os
import random
import re
import sys
import numpy as np
import pandas as pd

DAMPING = 0.85
SAMPLES = 10000

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

Now, take a look at the `main` function. It expects an argument, which will be the name of a directory of a corpus of web pages we’d like to compute PageRanks for(like `corpus0`, `corpus1` etc.). 

The `crawl` function takes that directory, parses all of the HTML files in the directory, and returns a dictionary representing the corpus. The keys in that dictionary represent pages (e.g., `"2.html"`), and the values of the dictionary are a set of all of the pages linked to by the key (e.g. `{"1.html", "3.html"}`).

In [3]:
def error(dir,d,s):
    corpus = crawl(dir)
    ranks1 = sample_pagerank(corpus, d, s)
    ranks2 = iterate_pagerank(corpus, d)
    error = 0
    for page in list(corpus.keys()):
        error += (ranks1[page] - ranks2[page])**2
    return error

In [4]:
def main(dir):
    corpus = crawl(dir)
    ranks1 = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks1):
        print(f"  {page}: {ranks1[page]:.4f}")
    ranks2 = iterate_pagerank(corpus, DAMPING)
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks2):
        print(f"  {page}: {ranks2[page]:.4f}")
    error = 0
    for page in list(corpus.keys()):
        error += (ranks1[page] - ranks2[page])**2
    print('\n')
    print(f"The squared error is {error}")

def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages

The `main` function then calls the `sample_pagerank` function, whose purpose is to estimate the PageRank of each page by sampling. The function takes as arguments the corpus of pages generated by crawl, as well as the damping factor and number of samples to use. Ultimately, `sample_pagerank` should return a dictionary where the keys are each page name and the values are each page’s estimated PageRank (a number between 0 and 1).

The `main` function also calls the `iterate_pagerank` function, which will also calculate PageRank for each page, but using the iterative formula method instead of by sampling. The return value is expected to be in the same format, and we would hope that the output of these two functions should be similar when given the same corpus!



## Enough Talk, Lets Code!

The basic architecture of what you're supposed to do is laid out for you, you may change add additional functions/re-define the pre-existing ones, as long as you do the same thing under the hood!

### Complete the implementation of the `transition_model` function

The `transition_model` should return a dictionary representing the probability distribution over which page a random surfer would visit next, given a corpus of pages, a current page, and a damping factor.

* The function accepts three arguments: `corpus`, `page`, and `damping_factor`.
  * The `corpus` is a Python dictionary mapping a page name to a set of all pages linked to by that page.
  * The `page` is a string representing which page the random surfer is currently on.
  * The `damping_factor` is a floating point number representing the damping factor to be used when generating the probabilities.
* The return value of the function should be a Python dictionary with one key for each page in the corpus. Each key should be mapped to a value representing the probability that a random surfer would choose that page next. The values in this returned probability distribution should sum to `1`.
  * With probability `damping_factor`, the random surfer should randomly choose one of the links from `page` with equal probability.
  * With probability `1 - damping_factor`, the random surfer should randomly choose one of all pages in the corpus with equal probability.
* For example, if the corpus were `{"1.html": {"2.html", "3.html"}, "2.html": {"3.html"}, "3.html": {"2.html"}}`, the page was `"1.html"`, and the `damping_factor` was `0.85`, then the output of `transition_model` should be `{"1.html": 0.05, "2.html": 0.475, "3.html": 0.475}`. This is because with probability `0.85`, we choose randomly to go from page 1 to either page 2 or page 3 (so each of page 2 or page 3 has probability `0.425` to start), but every page gets an additional `0.05` because with probability `0.15` we choose randomly among all three of the pages.
* If `page` has no outgoing links, then `transition_model` should return a probability distribution that chooses randomly among all pages with equal probability. (In other words, if a page has no links, we can pretend it has links to all pages in the corpus, including itself.)


In [5]:
def transition_model(corpus, page, d):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    list_keys = list(corpus.keys())
    n = len(list_keys)
    ans = dict()
    if(len(corpus[page]) == None):
        for i in range(n):
            ans[list_keys[i]] = 1.0/n
    else:
        for i in range(n):
            ans[list_keys[i]] = (1.0/n)*(1-d)
        
        conn = list(corpus[page])
        m = len(conn)
        
        for i in range(m):
            ans[conn[i]] += (1.0/m)*(d) 
        

    return ans
    


### Complete the `sample_pagerank` function

The `sample_pagerank` function should accept a corpus of web pages, a damping factor, and a number of samples, and return an estimated PageRank for each page.

* The function accepts three arguments: `corpus`, a `damping_factor`, and `n`.
  * The `corpus` is a Python dictionary mapping a page name to a set of all pages linked to by that page.
  * The `damping_factor` is a floating point number representing the damping factor to be used by the transition model.
  * `n` is an integer representing the number of samples that should be generated to estimate PageRank values.

* The return value of the function should be a Python dictionary with one key for each page in the corpus. Each key should be mapped to a value representing that page’s estimated PageRank (i.e., the proportion of all the samples that corresponded to that page). The values in this dictionary should sum to `1`.
* The first sample should be generated by choosing from a page at random.
* For each of the remaining samples, the next sample should be generated from the previous sample based on the previous sample’s transition model.
  * You will likely want to pass the previous sample into your `transition_model` function, along with the `corpus` and the `damping_factor`, to get the probabilities for the next sample.
  * For example, if the transition probabilities are `{"1.html": 0.05, "2.html": 0.475, "3.html": 0.475}`, then 5% of the time the next sample generated should be `"1.html"`, 47.5% of the time the next sample generated should be `"2.html"`, and 47.5% of the time the next sample generated should be `"3.html"`.
* You may assume that `n` will be at least `1`.

In [6]:
def normalise(arr):
    s = 0
    for a in arr:
        s += a
    for i in range(len(arr)):
        arr[i] /= s
    return arr

In [7]:
def sample_pagerank(corpus, d, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    
    ans = dict()
    
    list_keys = list(corpus.keys())
    #num = random.randint(0,len(list_keys)-1)
    
    coeff = np.zeros(len(list_keys))
    for i in range(len(coeff)):
        coeff[i] = 1/len(coeff)
    
    for i in range(n):

        coeff = normalise(coeff)
        
        num = np.random.choice(np.arange(0, len(coeff)), p = coeff)   
        
        for j in range(n//500):
            coeff[num] += 1
            tm = transition_model(corpus, list_keys[num], d)
            list_goto = list(tm.keys())
            prob = []
            for l in range(len(list_goto)):
                prob.append(tm[list_goto[l]])
            prob = normalise(prob)
            goto = np.random.choice(np.arange(0, len(list_goto)), p = prob)
            for l in range(len(list_keys)):
                if(list_goto[goto] == list_keys[l]):
                    num = l
    
    coeff = normalise(coeff) 
    
    for i in range(len(list_keys)):
        ans[list_keys[i]] = coeff[i]
        
    return ans

### Complete the `iterate_pagerank` function

The `iterate_pagerank` function should accept a corpus of web pages and a damping factor, calculate PageRanks based on the iteration formula described above, and return each page’s PageRank accurate to within `0.001`.

* The function accepts two arguments: `corpus` and `damping_factor`.
  * The `corpus` is a Python dictionary mapping a page name to a set of all pages linked to by that page.
  * The `damping_factor` is a floating point number representing the damping factor to be used in the PageRank formula.
* The return value of the function should be a Python dictionary with one key for each page in the corpus. Each key should be mapped to a value representing that page’s PageRank. The values in this dictionary should sum to `1`.
* The function should begin by assigning each page a rank of `1 / N`, where `N` is the total number of pages in the corpus.
* The function should then repeatedly calculate new rank values based on all of the current rank values, according to the PageRank formula in the “Background” section. (i.e., calculating a page’s PageRank based on the PageRanks of all pages that link to it).
  * A page that has no links at all should be interpreted as having one link for every page in the corpus (including itself).
* This process should repeat until no PageRank value changes by more than 0.001 between the current rank values and the new rank values.


In [8]:
def iterate_pagerank(corpus, d):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    list_keys = list(corpus.keys())
    n = len(list_keys)
    pr = dict()
    pr_dash = dict()
    
    for i in range(n):
        pr[list_keys[i]] = 1.0/n
        pr_dash[list_keys[i]] = 1.0/n
    
    place_holder = True
    

    while(place_holder):
        for i in range(n):
            conn = list(corpus[list_keys[i]])
            m = len(conn)
            s = 0
            
            if(m == 0):
                pr[list_keys[i]] = 1.0/n
                continue
                
            for k in range(m):
                num_link = len(corpus[conn[k]])
                s += pr[conn[k]]/num_link
            
            pr[list_keys[i]] = (1-d)/n + d*(s)
        
        for i in range(n):
            if(abs(pr_dash[list_keys[i]] - pr[list_keys[i]]) > 0.001):
                place_holder = False
                pr_dash[list_keys[i]] = pr[list_keys[i]]
       
    
    return pr

Now, you could look at the result of your work. If you did everything correct, then you should get the priorities of each HTML page for each page printed, and these numbers would look reasonable.


In [9]:
SAMPLES = 20000
main("corpus1")    # Either corpus1, corpus2 or corpus3

PageRank Results from Sampling (n = 20000)
  bfs.html: 0.0756
  dfs.html: 0.0024
  games.html: 0.3975
  minesweeper.html: 0.1976
  minimax.html: 0.0506
  search.html: 0.0769
  tictactoe.html: 0.1993
PageRank Results from Iteration
  bfs.html: 0.0619
  dfs.html: 0.1145
  games.html: 0.2036
  minesweeper.html: 0.1079
  minimax.html: 0.1484
  search.html: 0.1858
  tictactoe.html: 0.1710


The squared error is 0.08061904593384521


In [10]:
SAMPLES = 30000
main("corpus1") 

PageRank Results from Sampling (n = 30000)
  bfs.html: 0.1334
  dfs.html: 0.1167
  games.html: 0.1489
  minesweeper.html: 0.1314
  minimax.html: 0.1349
  search.html: 0.2680
  tictactoe.html: 0.0667
PageRank Results from Iteration
  bfs.html: 0.0619
  dfs.html: 0.1145
  games.html: 0.2036
  minesweeper.html: 0.1079
  minimax.html: 0.1484
  search.html: 0.1858
  tictactoe.html: 0.1710


The squared error is 0.026487077128133717


In [11]:
SAMPLES = 40000
main("corpus1") 

PageRank Results from Sampling (n = 40000)
  bfs.html: 0.1495
  dfs.html: 0.0759
  games.html: 0.2363
  minesweeper.html: 0.1494
  minimax.html: 0.1130
  search.html: 0.2139
  tictactoe.html: 0.0620
PageRank Results from Iteration
  bfs.html: 0.0619
  dfs.html: 0.1145
  games.html: 0.2036
  minesweeper.html: 0.1079
  minimax.html: 0.1484
  search.html: 0.1858
  tictactoe.html: 0.1710


The squared error is 0.025878410657132055


In [12]:
SAMPLES = 50000
main("corpus1") 

PageRank Results from Sampling (n = 50000)
  bfs.html: 0.1307
  dfs.html: 0.0599
  games.html: 0.2493
  minesweeper.html: 0.1401
  minimax.html: 0.0902
  search.html: 0.1908
  tictactoe.html: 0.1390
PageRank Results from Iteration
  bfs.html: 0.0619
  dfs.html: 0.1145
  games.html: 0.2036
  minesweeper.html: 0.1079
  minimax.html: 0.1484
  search.html: 0.1858
  tictactoe.html: 0.1710


The squared error is 0.015279488326998563


The Material was adapted from CS50.